# Importing Libraries

In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split

# Data Preprocessing

In [ ]:
filenames = os.listdir('dataset/images')

In [ ]:
print(filenames[0:5])

In [ ]:
labels_df = pd.read_csv('dataset/dataset_labels.csv')

In [ ]:
labels_df.shape

In [ ]:
labels_df.head()

In [ ]:
labels_df['label'].value_counts()

In [ ]:
# encoding labels

labels_dictionary = {'airplane':0, 'automobile':1, 'bird':2, 'cat':3, 'deer':4, 'dog':5, 'frog':6, 'horse':7, 'ship':8, 'truck':9}
labels = [labels_dictionary[i] for i in labels_df['label']]

In [ ]:
print(labels[0:5])

In [ ]:
id_list = list(labels_df['id'])

In [ ]:
print(id_list[0:5])

In [ ]:
# converting image into numpy array

path = 'dataset/images/'
data = []
for id in id_list:
    image = Image.open(path + str(id) + '.png')
    image = np.array(image)
    data.append(image)

In [ ]:
# converting image list and label list into numpy array

X = np.array(data)
y = np.array(labels)

In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
print(X_train.shape, y_train.shape)

In [ ]:
# feature scalling
X_train = X_train / 255
X_test = X_test / 255

In [ ]:
X_train

# Training Neural Network

In [ ]:
!pip install tensorflow-gpu

In [ ]:
from tensorflow.keras import Sequential, models, layers
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras import optimizers

In [ ]:
convolutional_base = ResNet50(weights = 'imagenet', include_top = False, input_shape = (256,256,3))
convolutional_base.summary()

In [ ]:
num_classes = 10
model = models.Sequential()
model.add(layers.UpSampling2D((2,2)))
model.add(layers.UpSampling2D((2,2)))
model.add(layers.UpSampling2D((2,2)))
model.add(convolutional_base)
model.add(layers.Flatten())
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
model.compile(optimizer=optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['acc'])

In [ ]:
history = model.fit(X_train, y_train, validation_split=0.1, epochs=1)

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print('Test Accuarcy = ', accuracy)

In [ ]:
# plot the loss value
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='validation loss')
plt.legend()
plt.show()

# plot the accuracy value
plt.plot(history.history['acc'], label='train accuracy')
plt.plot(history.history['val_acc'], label='validation accuracy')
plt.legend()
plt.show()